# Perceptrón multicapa

Después del perceptrón de Rosenblatt, durante muchos años se perdió el interés en las redes neuronales, debido a que nadie tenía una solución buena y computacionalmente coherente
para entrenarlas cuando estas tenían varias capas. Esto es hasta 1986, cuando se implementó el algoritmo de propagación hacia atrás (**backpropagation**). De este modo, el interés
en entender las redes neuronales multicapa se incrementó exponencialmente.

Un **perceptrón multicapa** es un modelo de red neuronal donde tenemos una capa de entrada y una de salida, al igual que en el modelo básico, pero además, tenemos **capas ocultas** en medio.
El propósito de estas **capas ocultas** es propagar los patrones en los datos de entrada a través de la red.

Hay que notar que añadir más capas ocultas no mejora necesariamente el desempeño de un modelo. Se trata de otro **hiperparámetro** a optimizar.

El proceso de entrenamiento en un perceptrón multicapa es sencillo:

1. Desde la capa de entrada, se propagan los datos hasta la capa de salida.
2. En base a la salida, calculamos el error en función de la **función de pérdida** que hayamos definido.
3. Aplicamos **propagación hacia atrás** para actualizar los pesos.

La forma de propagar hacia delante es sencilla, lo que te esperarías después de haber visto la lección anterior:

$z_{1}^{h} = x_{1}^{in} \cdot w_{1,1}^{h} + x_{2}^{in} \cdot w_{1,2}^{h} + ... + x_{n}^{in} \cdot w_{1,n}^{h}$

Donde $z_{1}^{h}$ es el **net_input** de la capa $h$, la neurona 1.

Ahora debemos calcular la **activación** de dicha neurona, que sería:

$a_{1}^{h} = \sigma (z_{1}^{h})$

Donde $\sigma$ representa la **función de activación**. Como vimos en la sesión pasada, la función de activación puede ser cualquiera que sea diferenciable y que se adapte a nuestras necesidades.
Por ejemplo, como ya vimos, si nuestras variables no son linealmente separables, una función sigmoide nos puede valer.

Debemos repetir este proceso para $a_{2}, a_{3}...$ y cualquier otra neurona que haya en la capa oculta.

Una vez tengamos la activación de cada neurona, deberemos ir a la siguiente capa y realizar el mismo proceso. En este caso, es una salida única con una sola neurona (puede haber más),
así que la operación sería:

$z_{out}^{h} = a_{1}^{h} \cdot w_{1}^{h} + a_{2}^{h} \cdot w_{2}^{h} + ... + x_{n}^{h} \cdot w_{n}^{h}$

$a_{out}^{h} = \sigma (z_{out}^{h})$

Una vez llegamos al final y tenemos nuestra predicción, la comparamos contra nuestra función de pérdida. Para ello, usaremos el **algoritmo de propagación hacia atrás**.

$\frac{\partial L}{\partial w_{1,1}^{out}} = \frac{\partial L}{\partial a_{1}^{out}} \cdot \frac{\partial a_{1}^{out}}{\partial z_{1}^{out}} \cdot \frac{\partial z_{1}^{out}}{\partial w_{1,1}^{out}}$

Este algoritmo se basa en la **regla de la cadena** para encontrar la derivada parcial de la función de perdida respecto del peso de la capa de salida.

Las matemáticas en este caso no son excesivamente importantes, sólo importa que entiendas el concepto básico: entrenar esto **para cada peso de la red** es muy complicado. Necesitaremos algo más potente que una CPU normal.

# Pytorch

Los ejemplos que hemos puesto arriba están simplificados y solo aplican a uno de los muchos caminos (aristas) que hay que recorrer en una red neuronal. Pero imagina por un segundo
si nuestra capa oculta tuviese más neuronas. Imagina si quisiésemos meter OTRA capa oculta. Los números escalan rápido. Esto en una CPU normal y corriente explota.

Para eso necesitamos **tarjetas gráficas**.

La ventaja que tiene una tarjeta gráfica respecto de una CPU normal es el **número de núcleos**. Cada núcleo de la GPU se puede considerar como un pequeño ordenador, a cada uno de los cuales se le puede mandar un proceso distinto haciendo computación en paralelo. Una CPU típica tiene cuatro u ocho núcleos. **Una GPU puede tener cientos o miles.** Imagina si cada una de las operaciones que he explicado arriba, cada uno de los *net inputs* de cada uno de los pesos, así como su respectiva función de activación, pudiesen calcularse en paralelo. Nos ahorraríamos muchísimo tiempo.
Redes neuronales que en una CPU tardarían semanas en entrenarse, en una GPU tardarían horas.

Sin embargo, hacer un uso eficiente de los recursos de una GPU no es una tarea tan sencilla, por no hablar de que la implementación de esta solución en paralelo es complicada. Existen paquetes
que nos permiten usar estos recursos, como por ejemplo CUDA, pero programar con CUDA es complicado.

Para eso sirve Pytorch.

Pytorch es un framework de deep learning escalable para implementar y ejecutar modelos de redes neuronales complejos de manera muchísimo más sencilla. Pytorch abstrae muchísimos de los conceptos
que hemos estado aprendiendo, como la función de activación o el *net_input*, para que nosotros, como científicos de datos, solo debamos preocuparnos por el modelo.

A continuación vamos a definir algunos conceptos de Pytorch importantes de entender para utilizarlo correctamente.

## [Tensores](https://pytorch.org/docs/stable/tensors.html)

Un tensor es una **generalización de un vector n-dimensional**. En la práctica, un tensor lo podemos usar para referirnos a un valor escalar (un 3, un 5...) o a un vector, o a una matriz..., y así sucesivamente. Son la estructura de datos por defecto de Pytorch y se utilizan en todas sus operaciones.

Un tensor puede "vivir" en la CPU o en la GPU. Podemos especificarlo con los métodos `.cpu()` y `.cuda()`.

Vamos a crear un tensor de ejemplo:

In [1]:
import torch

tensor_ejemplo = torch.rand(3, 3)
print("Tensor de ejemplo:")
print(tensor_ejemplo)

Tensor de ejemplo:
tensor([[0.3929, 0.1339, 0.8994],
        [0.0242, 0.6998, 0.0354],
        [0.1168, 0.6897, 0.5384]])


Veamos unas pocas operaciones que podemos realizar con él:

In [5]:
# podemos acceder a elementos concretos, como en una matriz
elemento = tensor_ejemplo[1, 1]
print(f"Elemento en la fila 1, columna 1: \n{elemento}\n\n")

# podemos hacer operaciones elemento a elemento, como
# con las matrices de numpy
tensor_a = torch.tensor([[1, 2], [3, 4]])
tensor_b = torch.tensor([[5, 6], [7, 8]])

suma = tensor_a + tensor_b
print(f"[+] Suma de tensores: \n{suma}\n\n")

# producto de tensores
producto = torch.matmul(tensor_a, tensor_b)
print(f"[*] Producto de tensores: \n{producto}\n\n")

# cambio de forma del tensor
tensor_reshape = tensor_a.view(1, 4)
print(f"[+] Nuevo tensor: \n{tensor_reshape}")

Elemento en la fila 1, columna 1: 
0.6998239159584045


[+] Suma de tensores: 
tensor([[ 6,  8],
        [10, 12]])


[*] Producto de tensores: 
tensor([[19, 22],
        [43, 50]])


[+] Nuevo tensor: 
tensor([[1, 2, 3, 4]])


La operación más interesante de cara a deep learning son las sumas-producto, o producto escalar (dot product en inglés):

In [8]:
dot_product = torch.dot(tensor_a.view(-1), tensor_b.view(-1))
print(f"[·] Producto escalar: {dot_product}\n\n")

[·] Producto escalar: 70




## [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

Un DataLoader en PyTorch es una clase que se utiliza para cargar datos en lotes de manera eficiente para su procesamiento en modelos de deep learning. Básicamente, se alimenta de un dataset y nos proporciona un iterador para ir iterando sobre este. El tamaño de cada iteración (o batch) lo podemos especificar en el constructor. [Mira este tutorial sobre cómo usarlo con los datasets](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

## [Clase "nn.Module"](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

Es la clase base para todos los modelos de redes neuronales. Las redes que creemos nosotros deben heredar de esta clase.

Por defecto, esta clase cuenta con un constructor, y un método `forward`. Ese método es donde se realizará la propagación hacia delante de la red.